In [1]:
import pandas as pd 
import numpy as np 
from datetime import datetime 
from sqlalchemy import create_engine
import pyodbc
import sys
import sqlite3

### Import file 

In [2]:
df_import = pd.read_csv('sales.csv')

In [3]:
df_import

,Date,Store_ID,Product_ID,Quantity_Sold,Sales_Amount
0,2024-08-01,1,P001,10,150.0
1,2024-08-01,1,P002,5,75.0
2,2024-08-01,2,P001,8,120.0
3,2024-08-02,1,P001,12,180.0


### Load into DATALAKE

In [4]:
conn_lake = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};' #look for ODBC in SQL Settings
    'SERVER=your_server_name;' # find the server name to input
    'DATABASE=your_database_name;' # the database
    'UID=your_username;' #the user account
    'PWD=your_password' #the password
)
cursor = conn_lake.cursor()

table_name_lake = 'DataLake'
cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {table_name_lake} (
        Date datetime,
        Store_ID INT,
        Product_ID VARCHAR(10),
        Quantity_Sold INT,
        Sales_Amount INT
    )
''')
df_import.to_sql(table_name_lake, conn_lake, if_exists= 'append', index = False)

NameError: name 'engine' is not defined

### Transform

In [ ]:
### Ensure datetime format :  YYYY-MM-DD format

df_import['Date'] = pd.to_datetime(df_import['Date'], format='%Y-%m-%d')

In [5]:
### Handling the missing value in the Quantity Sold and Sale Amount
column_have_mising = ['Quantity_Sold','Sales_Amount']
df_import[column_have_mising] = df_import[column_have_mising].fillna(0)

In [53]:
### Total sale of each store per day
df_import['Total_Sale'] = df_import['Quantity_Sold']*df_import['Sales_Amount']

In [54]:
### Sale per product
df_import['Sales_Per_Product'] = df_import['Sales_Amount']/df_import['Quantity_Sold']

### Load to DATAWAREHOUSE

In [ ]:
# Create database
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};' #look for ODBC in SQL Settings
    'SERVER=your_server_name;' # find the server name to input
    'DATABASE=your_database_name;' # the database
    'UID=your_username;' #the user account
    'PWD=your_password' #the password
)
cursor = conn.cursor()

table_name = 'DataWarehouse'
cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        Date datetime,
        Store_ID INT,
        Product_ID VARCHAR(10),
        Quantity_Sold INT,
        Sales_Amount INT
    )
''')
df_import.to_sql(table_name,conn, if_exists='append', index = False)
conn.commit()
conn.close()